In [2]:
import os
import time
import requests
import pandas as pd
from selenium import webdriver
import chromedriver_binary

In [3]:
# url = 'https://www.dmm.co.jp/digital/videoa/'
url = 'https://www.dmm.co.jp/litevideo/'
driver = webdriver.Chrome()
driver.get(url)
driver.find_element_by_link_text("はい").click()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  """


In [4]:
driver.get(url)
search_bar = driver.find_element_by_name("searchstr")
text = input('キーワードからさがす')
search_bar.send_keys(text)
search_bar.submit()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  


キーワードからさがす 佐知子　単体


In [5]:
def collect_search(name,url):
    data = {'Title': [],
        'Image':[],
        'Link':[]}
    df = pd.DataFrame(data)
    driver.get(url)
    ninzu = driver.find_element_by_class_name('list-boxcaptside.list-boxpagenation').text
    num = -(-int(ninzu.split("\u3000")[0][0:-5].replace(',',''))//30)
    for i in range(num):
        link = url+'/page='+str(i+1)+'/'
        driver.get(link)
        elements = driver.find_element_by_class_name('d-item').find_elements_by_tag_name('img')
        for element in elements:
            src = element.get_attribute('src')
            alt = element.get_attribute('alt')
            link = element.find_element_by_xpath('../..').get_attribute('href')
            se = pd.Series([alt,src,link],['Title','Image','Link'])
            df = df.append(se, ignore_index=True)
    df.to_csv('../Picture/data/'+name+'.csv',index=False)   

In [6]:
collect_search(text,driver.current_url)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  import sys
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  if sys.path[0] == '':
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/selenium/webdriver/remote/webelement.py:358: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_elements_by_* commands are deprecated. Please use find_elements() instead")
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/selenium/webdriver/remote/webelement.py:392: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_element_by_* c

In [7]:
file = f'data/{text}.csv'
df = pd.read_csv(file)
folders = os.path.basename(file).split('.')
os.makedirs('movie/'+folders[0])

In [8]:
links = df['Link'].tolist()
for i,link in enumerate(links):
    driver.get(link)
    iframes = driver.find_elements_by_tag_name('iframe')
    if len(iframes) < 2:
        continue
    src = iframes[1].get_property('src')
    driver.get(src)
    src = driver.find_element_by_tag_name('video').get_attribute('src')
    name = os.path.basename(src)
    res = requests.get(src)
    with open('movie/'+folders[0]+'/'+name, 'wb') as f:
        f.write(res.content)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  if __name__ == '__main__':


In [9]:
driver.quit()